In [1]:
import train
import coco_dataset
import transforms as T

main_folder_path = "../"
model_name = "tf_efficientdet_d0"
target_dim = 512

print("Memory usage [{}]".format(train.get_memory_usage()))
num_classes, train_df, test_df, categories_df = train.process_data_coco(main_folder_path, None)

dataset = coco_dataset.COCODataset(train_df, True, model_name, main_folder_path, target_dim, num_classes,
                                   T.get_transform(train=True), False, False)
# dataset_test = coco_dataset.COCODataset(test_df, False, model_name, main_folder_path, target_dim, num_classes,
#                                              T.get_transform(train=True), False)
print("Memory usage [{}]".format(train.get_memory_usage()))

Running on non Google Colab env
Memory usage [32.6]
loading annotations into memory...
Done (t=11.81s)
creating index...
index created!
loading annotations into memory...
Done (t=0.37s)
creating index...
index created!
Num of classes is [90]
[{'supercategory': 'person', 'id': 1, 'name': 'person'}, {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}, {'supercategory': 'vehicle', 'id': 3, 'name': 'car'}, {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'}, {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'}, {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'}, {'supercategory': 'vehicle', 'id': 7, 'name': 'train'}, {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'}, {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'}, {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'}, {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'}, {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'}, {'supercategory': 'outdoor', 'id': 14, 'name': 'parking me

In [3]:
import os
import torch
import utils
import time
import engine

#os.environ['LRU_CACHE_CAPACITY'] = '1'  # I don't think this helped

device = 'cuda:0'
cpu_device = 'cpu'

data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=7, shuffle=False, num_workers=1,
            collate_fn=utils.collate_fn)

model = train.get_model_instance_segmentation_efficientnet(model_name, num_classes, target_dim, freeze_batch_norm=True)
# model = train.get_model_instance_segmentation(num_classes)
model.to(device)
model.eval()
with torch.no_grad():
    engine.evaluate(model, data_loader, device, box_threshold=0.0001)  
print("Memory usage after loop [{}]".format(train.get_memory_usage()))
            
            
# print("Memory usage at iteration [{}] is [{}]".format(0, train.get_memory_usage()))
# for i in range(20):
#     show_image_data_ground_truth(dataset, i, False)
# #     img, tgt = dataset.__getitem__(i)
#     print("Memory usage at iteration [{}] is [{}]".format(i, train.get_memory_usage()))

Using EffDet detection model
Freezing batch normalization weights
Memory usage 1 [70.4]
creating index...
index created!
Memory usage 2 [70.3]
Memory usage 3 [70.3]
Memory usage 4 [71.5]
Memory usage 5 [72.0]
Memory usage 6 [76.6]
Memory usage 7 [76.6]
Memory usage 8 [77.0]
Memory usage 9 [77.0]
---
Test:  [ 0/72]  eta: 0:02:13  model_time: 0.8030 (0.8030)  evaluator_time: 0.4878 (0.4878)  time: 1.8534  data: 0.5409  max mem: 952
Memory usage 4 [76.7]
Memory usage 5 [77.1]
Memory usage 6 [81.5]
Memory usage 7 [81.5]
Memory usage 8 [81.8]
Memory usage 9 [81.8]
---
Memory usage 4 [81.6]
Memory usage 5 [81.8]
Memory usage 6 [83.1]
Memory usage 7 [79.3]
Memory usage 8 [79.8]
Memory usage 9 [79.8]
---
Memory usage 4 [79.6]
Memory usage 5 [79.8]
Memory usage 6 [82.9]
Memory usage 7 [81.6]
Memory usage 8 [81.6]
Memory usage 9 [81.6]
---
Memory usage 4 [81.3]
Memory usage 5 [81.5]
Memory usage 6 [83.4]
Memory usage 7 [80.3]
Memory usage 8 [80.3]
Memory usage 9 [80.3]
---
Memory usage 4 [80.1]


In [1]:
import torch
import train
import visualize

vis = visualize.Visualize(main_folder_path, categories_df, target_dim, dest_folder=None)

def show_image_data_ground_truth(dataset, idx, is_colab, figsize=(40, 40)):
    image, target = dataset.__getitem__(idx)
    print(len(target['labels']))
    vis.show_image_data(image, torch.sub(target['labels'], 1), target['masks'], target['boxes'], figsize=figsize)

print("Memory usage at iteration [{}] is [{}]".format(0, train.get_memory_usage()))
for i in range(150,200):
    show_image_data_ground_truth(dataset, i, False)
    print("Memory usage at iteration [{}] is [{}]".format(i, train.get_memory_usage()))

Running on non Google Colab env


NameError: name 'main_folder_path' is not defined

In [35]:
import torch
import gc
print("Memory usage [{}]".format(train.get_memory_usage()))
torch.cuda.empty_cache()
gc.collect()
print("Memory usage [{}]".format(train.get_memory_usage()))

Memory usage [61.5]
Memory usage [61.5]


In [2]:
import train
from PIL import Image
from pycocotools.coco import COCO

main_folder_path = '..'
train_annot_path = main_folder_path + '/Data/annotations/instances_train2017.json'
coco_train = COCO(train_annot_path)
image_ids = [id for id in coco_train.getImgIds() if len(coco_train.getAnnIds(imgIds=id, iscrowd=None)) > 0][:1000] 
for idx in range(1000):
    image_id = image_ids[idx]
    img = coco_train.loadImgs(image_id)[0]
    annIds = coco_train.getAnnIds(imgIds=img['id'], iscrowd=None)
    anns = coco_train.loadAnns(annIds)
    labels = torch.tensor([int(ann['category_id']) for ann in anns], dtype=torch.int64)
    print("Memory usage [{}]".format(train.get_memory_usage()))
    if train.get_memory_usage() > 80.0:
        print("Memory usage too high! [{}]".format(train.get_memory_usage()))
        break
print("Memory usage outside the loop [{}]".format(train.get_memory_usage()))

Running on non Google Colab env
loading annotations into memory...
Done (t=11.40s)
creating index...
index created!
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memory usage [40.7]
Memo

In [5]:
image, target = dataset.__getitem__(0)
print(target)

{'labels': tensor([4, 1, 1, 2]), 'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8), 'boxes': tensor([[136., 229., 224.

In [5]:
import utils
import time
import torch
from coco_eval import CocoEvaluator
from coco_utils import get_coco_api_from_dataset
from sys import getsizeof

device = 'cuda:0'
cpu_device = 'cpu'
data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=7, shuffle=False, num_workers=2,
            collate_fn=utils.collate_fn)

model = train.get_model_instance_segmentation_efficientnet(model_name, num_classes, target_dim, freeze_batch_norm=True)
model.to(device)
model.eval()


n_threads = torch.get_num_threads()
# FIXME remove this and make paste_masks_in_image run on the GPU
torch.set_num_threads(1)
cpu_device = torch.device("cpu")
model.eval()
metric_logger = utils.MetricLogger(delimiter="  ")
header = 'Test:'

coco = get_coco_api_from_dataset(data_loader.dataset, 0.001)
iou_types = ["bbox", "segm"]
coco_evaluator = CocoEvaluator(coco, iou_types)

with torch.no_grad():
    for images, targets in data_loader:
        images_gpu = list(img.to(device) for img in images)
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images_gpu, 0.001)

        # targets_cpu = [{k: v.to(cpu_device) if torch.is_tensor(v) else v for k, v in t.items()} for t in targets]
        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time
            
        # targets_cpu = [{k: v.to(cpu_device) if torch.is_tensor(v) else v for k, v in t.items()} for t in targets]
        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"]: output for target, output in zip(targets, outputs)}  # ofekp: this used to be target["image_id"].item()
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
#         metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)
        
        torch.cuda.empty_cache()
        print("Memory usage [{}] coco_evaluator [{}]".format(train.get_memory_usage(), getsizeof(coco_evaluator)))
        if train.get_memory_usage() > 80.0:
            print("Memory usage too high! [{}]".format(train.get_memory_usage()))
            break
            
# gather the stats from all processes
# metric_logger.synchronize_between_processes()
print("Averaged stats:", metric_logger)
coco_evaluator.synchronize_between_processes()

# accumulate predictions from all images
coco_evaluator.accumulate()
coco_evaluator.summarize()
torch.set_num_threads(n_threads)

Using EffDet detection model
Freezing batch normalization weights
creating index...
index created!
Memory usage [69.5] coco_evaluator [48]
Memory usage [69.9] coco_evaluator [48]
Memory usage [70.5] coco_evaluator [48]
Memory usage [70.6] coco_evaluator [48]
Memory usage [70.8] coco_evaluator [48]
Memory usage [71.8] coco_evaluator [48]
Memory usage [72.1] coco_evaluator [48]
Memory usage [72.6] coco_evaluator [48]
Memory usage [72.9] coco_evaluator [48]
Memory usage [73.3] coco_evaluator [48]
Memory usage [74.2] coco_evaluator [48]
Memory usage [74.3] coco_evaluator [48]
Memory usage [75.1] coco_evaluator [48]
Memory usage [75.8] coco_evaluator [48]
Memory usage [76.1] coco_evaluator [48]
Memory usage [76.6] coco_evaluator [48]
Memory usage [76.9] coco_evaluator [48]
Memory usage [77.2] coco_evaluator [48]
Memory usage [78.2] coco_evaluator [48]
Memory usage [78.8] coco_evaluator [48]
Memory usage [79.8] coco_evaluator [48]
Memory usage [80.3] coco_evaluator [48]
Memory usage too high